In [1]:
import numpy as np 
import pandas as pd
import torch 
from Dataset.Utils import prepare_data_range


In [2]:
embeds_paths =  [
      "/mnt/d/ML/Kaggle/CAFA6-new/Dataset/esm2_embeds_cafa5/train_embeddings.npy",
      "/mnt/d/ML/Kaggle/CAFA6-new/Dataset/protBERT_embeddings/train_embeddings.npy",
      "/mnt/d/ML/Kaggle/CAFA6-new/Dataset/t5_embeds/train_embeds.npy"
    ]
ids_paths = [
      "/mnt/d/ML/Kaggle/CAFA6-new/Dataset/esm2_embeds_cafa5/train_ids.npy",
      "/mnt/d/ML/Kaggle/CAFA6-new/Dataset/protBERT_embeddings/train_ids.npy",
      "/mnt/d/ML/Kaggle/CAFA6-new/Dataset/t5_embeds/train_ids.npy"
    ]

In [5]:
embeds_df_list = []

for i in range(len(embeds_paths)):
    embeds = np.load(embeds_paths[i], allow_pickle=True)
    ids = np.load(ids_paths[i], allow_pickle=True)
    print(f"Embeds shape: {embeds.shape}, Ids shape: {ids.shape}")
    embeds_df = pd.DataFrame({"EntryID": ids, "embed": list(embeds)})
    embeds_df_list.append(embeds_df)

Embeds shape: (142246, 1280), Ids shape: (142246,)
Embeds shape: (142246, 1024), Ids shape: (142246,)
Embeds shape: (142246, 1024), Ids shape: (142246,)
Embeds shape: (142246, 1024), Ids shape: (142246,)
Embeds shape: (142246, 1024), Ids shape: (142246,)


In [4]:
embeds_df_list = None
#collect garbage 
import gc
gc.collect()

44

In [4]:
import sys
#size in Mb
sys.getsizeof(embeds_df_list[1]) / (1024 * 1024)

23.778234481811523

In [6]:
embeds_df_list[2]['embed'][0]

array([ 0.04948843, -0.03293516,  0.03247323, ..., -0.04353154,
        0.0964628 ,  0.07306959], shape=(1024,))

In [6]:
embeds_df = None

def concat_embeds(row):
    embed1 = row['concat_embed']
    embed2 = row['embed']
    concat_embed = np.concatenate((embed1, embed2))
    return concat_embed

for i in range(1, len(embeds_df_list)):
    print(f"Embeds DF {i} shape: {embeds_df_list[i].shape}")
    if embeds_df is None:
        embeds_df = embeds_df_list[i-1]
        embeds_df_list[i-1] = None
        embeds_df.rename(columns={"embed": 'concat_embed'}, inplace=True)
    embeds_df = pd.merge(embeds_df, embeds_df_list[i], on="EntryID", how='inner')
    #delete merged df from list to save memory
    embeds_df_list[i] = None
    #concat embeddings
    # embeds_df['concat_embed'] = embeds_df.apply(concat_embeds, axis=1)
    # embeds_df.drop(columns=['embed'], inplace=True)

embeds_df_list = None
gc.collect()
embeds_df

Embeds DF 1 shape: (142246, 2)
Embeds DF 2 shape: (142246, 2)


,EntryID,concat_embed,embed_x,embed_y
0,Q9ZSA8,"[-0.092291094, -0.066283956, -0.01226195, 0.04...","[0.10063905, 0.019749707, 0.09575205, -0.19726...","[0.06355330348014832, 0.092497318983078, -0.02..."
1,P25353,"[0.011624349, -0.030317612, -0.0058019715, 0.0...","[0.13231392, 0.06233021, 0.094939865, -0.14466...","[0.011784744448959827, -0.04589618742465973, 0..."
2,A0A2R8YCW8,"[0.02737274, -0.041047025, -0.029205365, 0.028...","[0.14164767, 0.051090643, 0.0756673, -0.143788...","[0.030849341303110123, 0.03080468438565731, 0...."
3,G3V5N8,"[0.033766113, -0.07888931, -0.05974137, 0.0456...","[0.13703091, 0.12680532, 0.117955275, -0.16379...","[-0.013897695578634739, 0.0360458567738533, 0...."
4,A0A140LFN4,"[0.0119482, -0.002107593, -0.084922194, 0.0687...","[0.061685473, 0.050877817, 0.064081304, -0.150...","[-0.016684038564562798, 0.09142760932445526, 0..."
...,...,...,...,...
142241,O81299,"[0.022891823, -0.06792282, -0.057677716, -0.03...","[0.14427365, 0.024739401, 0.094333366, -0.1877...","[0.04659712687134743, 0.09900099784135818, 0.0..."
142242,Q55AH8,"[0.014263509, 0.033850852, 0.025951566, 0.0654...","[0.15403968, 0.03255585, 0.080744386, -0.14291...","[0.0451243557035923, -0.04376251623034477, 0.0..."
142243,Q80VK8,"[0.03865814, 0.054107588, 0.07705976, 0.026639...","[0.14926998, 0.024992127, 0.07972684, -0.18500...","[0.00606992281973362, -0.05574773624539375, -0..."
142244,Q8IS12,"[0.041082174, -0.020872245, 0.00023603393, 0.0...","[0.120118834, 0.10417359, 0.10750137, -0.15010...","[0.13251236081123352, 0.08502178639173508, 0.0..."


In [7]:
em = embeds_df['embed_y'].to_list()
em = np.stack(em)  

In [9]:
import sys
sys.getsizeof(em) / (1024 * 1024)

1111.2969970703125

In [10]:
em = None
gc.collect()

1032

In [11]:
#convert embeds column to np array
cols = ['concat_embed', 'embed_x', 'embed_y']
embeds =None
for col in cols:
    em = embeds_df[col].to_list()
    em = np.stack(em)   
    if embeds is None:
        embeds = em 
    else:
        embeds = np.concatenate([embeds, em], axis=1)
    em = None
    gc.collect()

: 

In [14]:
embeds2 = embeds_df['embed_x'].to_list()
embeds2 = np.stack(embeds2)
embeds2

array([[ 0.10063905,  0.01974971,  0.09575205, ..., -0.04837033,
        -0.01848502,  0.02100725],
       [ 0.13231392,  0.06233021,  0.09493987, ..., -0.08690946,
         0.00162027, -0.00218274],
       [ 0.14164767,  0.05109064,  0.0756673 , ..., -0.09845898,
        -0.03267479,  0.00680899],
       ...,
       [ 0.14926998,  0.02499213,  0.07972684, ..., -0.07171173,
        -0.03229819,  0.0570601 ],
       [ 0.12011883,  0.10417359,  0.10750137, ..., -0.07582046,
         0.00545941, -0.00694445],
       [ 0.02316349,  0.01070298,  0.07081697, ..., -0.01841847,
        -0.04550426,  0.05292074]], shape=(142246, 1024), dtype=float32)

In [15]:
sys.getsizeof(embeds2) / (1024 * 1024)

555.6485595703125

In [11]:
ids_list[1]

array(['P20536', 'O73864', 'O95231', ..., 'Q5RGB0', 'A0A2R8QMZ5',
       'A0A8I6GHU0'], shape=(142246,), dtype='<U10')

In [3]:
train_embeds = np.load('/mnt/d/ML/Kaggle/CAFA6-new/Dataset/t5_embeds/train_embeds.npy')
train_ids = np.load('/mnt/d/ML/Kaggle/CAFA6-new/Dataset/t5_embeds/train_ids.npy')

In [5]:
train_embeds.shape

(142246, 1024)

In [4]:
embeds_df = pd.DataFrame({"EntryID": train_ids, "embed": list(train_embeds)})
embeds_df = embeds_df.set_index('EntryID')
# print(embeds_df['EntryID'].nunique())
embeds_df

,embed
EntryID,
Q9ZSA8,"[-0.092291094, -0.066283956, -0.01226195, 0.04..."
P25353,"[0.011624349, -0.030317612, -0.0058019715, 0.0..."
A0A2R8YCW8,"[0.02737274, -0.041047025, -0.029205365, 0.028..."
G3V5N8,"[0.033766113, -0.07888931, -0.05974137, 0.0456..."
A0A140LFN4,"[0.0119482, -0.002107593, -0.084922194, 0.0687..."
...,...
O81299,"[0.022891823, -0.06792282, -0.057677716, -0.03..."
Q55AH8,"[0.014263509, 0.033850852, 0.025951566, 0.0654..."
Q80VK8,"[0.03865814, 0.054107588, 0.07705976, 0.026639..."


In [5]:
train_terms_df = pd.read_csv("/mnt/d/ML/Kaggle/CAFA6/cafa-6-protein-function-prediction/Train/train_terms.tsv",
                    delimiter='\t')
train_terms_df

,EntryID,term,aspect
0,Q5W0B1,GO:0000785,C
1,Q5W0B1,GO:0004842,F
2,Q5W0B1,GO:0051865,P
3,Q5W0B1,GO:0006275,P
4,Q5W0B1,GO:0006513,P
...,...,...,...
537022,Q06667,GO:0070481,P
537023,B1NF19,GO:0033075,P
537024,B1NF19,GO:0047052,F
537025,B1NF19,GO:0047056,F


In [6]:
data = prepare_data_range(train_terms_df, embeds_df, [0, 64])

In [7]:
import torch
import torch.nn as nn
from Utils.tokenizer import nearly_orthogonal_vectors

class EmbedTokenizer(nn.Module):
    def __init__(self, D, d, N, rng=None):  
        super(EmbedTokenizer, self).__init__()
        """
        Parameters
        ----------
        D : int
            Dimension of the embedding space.
        d : int
            Dimension of the token space.
        N : int
            Number of tokens.
        rng : np.random.Generator or None
            Random generator. If None, use default.
        """
        if rng is None:
            rng = np.random.default_rng()

        K = D
        V = nearly_orthogonal_vectors(D, K, tol=0.01, max_tries=100, rng=rng)
        
        P = []

        for i in range(N):
            indices = np.arange(K)
            sampled_idx = np.random.choice(indices, size=d, replace=False)
            p = V[sampled_idx]
            P.append(torch.tensor(p, dtype=torch.float32))
        
        # Stack all P matrices into a single tensor: (N, d, D)
        self.P = torch.stack(P)
        self.register_buffer('P_buffer', self.P)

    def forward(self, x):

        """
        Parameters
        ----------
        x : Tensor, shape (batch_size, D) or (D,)
            Input embeddings.

        Returns
        -------
        tokens : Tensor, shape (batch_size, N, d) or (N, d)
            Token representations.
        """
        # Handle both single sample and batch
        squeeze_output = False
        if x.dim() == 1:
            x = x.unsqueeze(0)  # (1, D)
            squeeze_output = True
        
        batch_size, D = x.shape
        
        # Get P and move to correct device/dtype
        P = self.P_buffer.to(dtype=x.dtype, device=x.device)  # (N, d, D)

        # Reshape P to 2D: (D, N*d) and compute in one matmul to ensure batch dim is first
        P_2d = P.permute(2, 0, 1).reshape(D, -1)  # (D, N*d)
        tokens = torch.matmul(x, P_2d)  # (batch_size, N*d)
        tokens = tokens.reshape(batch_size, P.shape[0], P.shape[1])  # (batch_size, N, d)

        if squeeze_output:
            tokens = tokens.squeeze(0)  # Remove batch dimension if input was 1D
        
        return tokens


In [8]:
from torch.utils.data import Dataset, DataLoader
import numpy as np


class EmbeddingsDataset():
    def __init__(self, data, oversample_indices=None):
        self.data = data
        self.oversample_indices = oversample_indices if oversample_indices is not None else list(range(len(self.data['embeds'])))

    def __len__(self):
        return len(self.oversample_indices)

    def __getitem__(self, idx):
        sample_idx = self.oversample_indices[idx]

        return {
            'entryID': self.data['entries'][sample_idx],
            'embeds' : self.data['embeds'][sample_idx],
            'label' : self.data['labels'][sample_idx]
        }

In [10]:
# Tokenized dataset + collate that runs embeddings through the tokenizer
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import time

class TokenizedEmbeddingsDataset(Dataset):
    """Dataset that yields raw embeddings; tokenization is done in collate_fn for batching."""
    def __init__(self, data, oversample_indices=None):
        self.data = data
        self.oversample_indices = oversample_indices if oversample_indices is not None else list(range(len(self.data['embeds'])))

    def __len__(self):
        return len(self.oversample_indices)

    def __getitem__(self, idx):
        sample_idx = self.oversample_indices[idx]
        return {
            'entryID': self.data['entries'][sample_idx],
            'embed': self.data['embeds'][sample_idx],
            'label': self.data['labels'][sample_idx]
        }


def collate_tokenize(batch, tokenizer, device=None, dtype=torch.float32):
    """Batch raw embeds, move to device, run tokenizer once for the batch.

    Returns a dict with keys: 'entryID' (list), 'tokens' (Tensor: batch,N,d), 'label' (Tensor)
    """
    embeds = [item['embed'] for item in batch]
    # stack into (batch_size, D)
    embeds = np.stack(embeds)
    embeds = torch.tensor(embeds, dtype=dtype)
    if device is not None:
        embeds = embeds.to(device)

    # ensure tokenizer is on the requested device
    try:
        tokenizer_device = next(tokenizer.parameters()).device
    except StopIteration:
        # tokenizer may have no parameters; attempt P_buffer or default to cpu
        tokenizer_device = getattr(tokenizer, 'P_buffer', torch.tensor(0)).device if hasattr(tokenizer, 'P_buffer') else torch.device('cpu')

    if device is not None and tokenizer_device != device:
        try:
            tokenizer = tokenizer.to(device)
        except Exception:
            pass

    with torch.no_grad():
        tokens = tokenizer(embeds)  # expected (batch_size, N, d)

    # Fix swapped axes if tokenizer produced (N, batch_size, d)
    try:
        if hasattr(tokenizer, 'P_buffer') and isinstance(tokenizer.P_buffer, torch.Tensor):
            N = tokenizer.P_buffer.shape[0]
            if tokens.dim() == 3 and tokens.shape[0] == N and tokens.shape[1] != N:
                tokens = tokens.permute(1, 0, 2).contiguous()
    except Exception:
        pass

    # Convert labels to numpy array first to avoid slow list->tensor warning
    labels_np = np.array([item['label'] for item in batch])
    labels = torch.tensor(labels_np, dtype=torch.long)

    entryIDs = [item['entryID'] for item in batch]

    return {'entryID': entryIDs, 'tokens': tokens, 'label': labels}


class PrefetchLoader:
    """DataLoader wrapper that asynchronously preloads next batch to CUDA.

    Use like: prefetch_loader = PrefetchLoader(loader, device)
    for batch in prefetch_loader:
        # batch already moved to device (if CUDA)

    Notes:
    - Works only when `device` is CUDA; otherwise yields batches unchanged.
    - Requires that collate_fn returns torch tensors (tokens/label).
    """
    def __init__(self, loader, device):
        self.loader = loader
        self.device = device

    def __iter__(self):
        self.loader_iter = iter(self.loader)
        if self.device.type == 'cuda':
            self.stream = torch.cuda.Stream()
            self._preload()
        else:
            self.next_batch = None
        return self

    def _preload(self):
        try:
            self.next_batch = next(self.loader_iter)
        except StopIteration:
            self.next_batch = None
            return
        # move tensors to device asynchronously
        if self.device.type == 'cuda':
            with torch.cuda.stream(self.stream):
                for k, v in list(self.next_batch.items()):
                    if isinstance(v, torch.Tensor):
                        # non_blocking requires pinned memory on CPU side
                        # If tensor already on CUDA, .cuda() will be a no-op on same device
                        self.next_batch[k] = v.cuda(non_blocking=True)

    def __next__(self):
        if self.device.type == 'cuda':
            if self.next_batch is None:
                raise StopIteration
            # wait for copy stream
            torch.cuda.current_stream().wait_stream(self.stream)
            batch = self.next_batch
            # preload next
            self._preload()
            return batch
        else:
            # CPU path: just yield from underlying loader
            return next(self.loader_iter)


# Example usage:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# If `tokenizer` is not defined in the notebook, try to instantiate one from the class
if 'tokenizer' not in globals():
    try:
        D = int(np.asarray(data['embeds'][0]).shape[0])
    except Exception:
        raise RuntimeError('Cannot determine embed dimension from `data`. Make sure `data` is prepared.')
    try:
        # try to instantiate EmbedTokenizer if available
        tokenizer = EmbedTokenizer(D=D, d=256, N=64)
        print('Created EmbedTokenizer with D=', D)
    except Exception:
        # fallback simple tokenizer
        class SimpleTokenizer(torch.nn.Module):
            def __init__(self, D, d=256, N=100):
                super().__init__()
                # random projection stored as buffer
                proj = torch.randn(N, d, D)
                self.register_buffer('proj', proj)
            def forward(self, x):
                return torch.matmul(x, self.proj.transpose(-1, -2))
        tokenizer = SimpleTokenizer(D)
        print('Using fallback SimpleTokenizer')

# Move tokenizer to device
try:
    tokenizer = tokenizer.to(device)
except Exception:
    pass

# Build DataLoader: do NOT pin_memory because collate moves tensors to device already
dataset = TokenizedEmbeddingsDataset(data)
loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0, pin_memory=False, collate_fn=lambda b: collate_tokenize(b, tokenizer, device))

# Wrap with PrefetchLoader when using CUDA
if device.type == 'cuda':
    prefetch_loader = PrefetchLoader(loader, device)
    iter_loader = iter(prefetch_loader)
else:
    iter_loader = iter(loader)

# quick sanity check & timing
start = time.time()
try:
    batch = next(iter_loader)
    elapsed = time.time() - start
    print('tokens shape:', batch['tokens'].shape)
    print('labels shape :', batch['label'].shape)
    print(f'time (single batch): {elapsed:.4f}s')
except Exception as e:
    print('Error when iterating loader:', e)


Using device: cuda
tokens shape: torch.Size([32, 64, 256])
labels shape : torch.Size([32, 64])
time (single batch): 0.0079s


In [19]:
num_classes = 64
hidden_dim = 256

C_in = tokenizer.P.shape[1]  # token dimension d

model = Query2Label(
    num_classes=num_classes,
    in_dim=C_in,
    nheads=8,
    num_encoder_layers=1,
    num_decoder_layers=2,
    dim_feedforward=2048,
    dropout=0.1,
)

model = model.to(device)
# quick forward pass test

batch = next(iter_loader)

with torch.no_grad():
    logits = model(batch['tokens'])
    print('logits shape:', logits.shape)  # expected (B, num_classes)


logits shape: torch.Size([32, 64])
